In [29]:
import pandas as pd
import numpy as np
import importlib
import sys
import types
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, balanced_accuracy_score
import matplotlib.pyplot as plt
from fractions import Fraction
import json
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.naive_bayes import GaussianNB 
from sklearn.naive_bayes import BernoulliNB 

# load the data
file_path_diet = "diet.csv"
file_path_reviews = "reviews.csv"
file_path_requests = "requests.csv"
file_path_recipes = "recipes.csv"

df_diet = pd.read_csv(file_path_diet)
df_reviews = pd.read_csv(file_path_reviews)
df_requests = pd.read_csv(file_path_requests)
df_recipes = pd.read_csv(file_path_recipes)
df_testset_reviews = None
df_trainset_reviews = None
df_diet_clean = None
df_recipes_clean = None
df_requests_clean = None
df_trainset_clean =None
df_testset_clean =None

#-------------- Hilfsfunktionen Data Cleaning----------------------------------
def erstelle_Meal_Typen():
    global df_recipes
    #Bringe Zutaten in Listenform
    df_recipes["RecipeIngredientParts"]= df_recipes["RecipeIngredientParts"].apply(extract_list_elements)
    
    #Kategorisiere Rezepte
    df_recipes['Type_Meal'] = df_recipes['RecipeIngredientParts'].apply(categorize_recipe)

def extract_list_elements(s):
    cleaned_string = s.replace('\\"', '')
    cleaned_string = cleaned_string.replace('c(', '[')
    cleaned_string = cleaned_string.replace(')', ']')
    if s.find("c(") != -1:
        return json.loads(cleaned_string)
    else:
        return []

def categorize_recipe(ingredients):
    #Definiere Omnivore und Vegetarische Keywords sowie Exceptions
    omnivor_keywords = set(["chicken", "beef", "pork", "lamb","turkey", "duck", "goat","bacon", "ham",
                            "meat","sausage", "salami", "hot dog","salmon", "tuna", "trout", "lobster", 
                            "cod", "shrimp","crab", "oyster","squid", "octopus", "liver", "heart", "tongue", 
                            "steak", "sea bass", "clams", "halibuts", "fillet", "scallops", "fish",]) 
    vegetarian_keywords = set(["milk", "cheese", "butter", "yogurt", "cream", "egg", "honey", "gelatin", 
                               "whey", "casein", "lactose", "dairy", "mayonnaise", "mozzarella"])
    vegetarian_exceptions = set(["soymilk", "almond butter", "peanut butter", "vegan butter","coconut milk", 
                                 "coconut butter", "cashew butter"])
    #Durchsuche und Klassifiziere jedes Rezept
    for ingredient in ingredients:
        if any(omnivore_keyword in ingredient for omnivore_keyword in omnivor_keywords):
            return 'Omnivor'
        elif any(vegetarian_keyword in ingredient for vegetarian_keyword in vegetarian_keywords):
            if not any(exception in ingredient for exception in vegetarian_exceptions):
                return 'Vegetarisch'
    return 'Vegan'

def translate_recipe_yields_to_categories(s):
    if s == "Undefined":
        return "Undefined"
    result_list = s.split()
    for value in result_list[:1]:
        # print(value)
        if value.find("-") != -1:
            value = s.split("-")[0]
            # print("update")
            # print(value)
        fraction_obj = Fraction(value)
        val_int: float = float(fraction_obj)
        if val_int == 1:
            return "Single Portion"
        elif val_int == 2:
            return "Two Portions"
        elif val_int <= 5:
            return "Medium Portions"
        elif val_int > 5:
            return "Many Portions"
        else:
            return "error"



#-------------- Cleanen Datasets -------------------------#


#Clean Diet.csv
# --> Es gibt 271.907 rows und keine fehlenden Werte (ausser bei Diet 1)
# --> Es gibt 78.626 Omnivore, 49.897 Vegan, 143.383 Vegetarian User
# --> Es gibt keine Outlier bei Age, Diet-Typ ist gleichverteilt ueber Age
def clean_diet():
    global df_diet
    global df_diet_clean
    df_diet["AuthorId"] = df_diet["AuthorId"].astype("category")
    df_diet["Diet"] = df_diet["Diet"].astype("category")
    NAValues_diet = df_diet["Diet"].isna()
    df_diet = df_diet.loc[~NAValues_diet]
    df_diet_clean =df_diet

#1. Clean Reviews.csv 
# --> Es gibt 140.195 rows, Rating, Like und TestSetID haben fehlende Werte
# --> Immer wenn Rows im TestSet sind, haben sie keinen Like Wert: 97381 + 42814 = 140195
# --> Rating ist immer 2.0 ansonsten nicht vorhanden
#2. Erstelle Test und Training Set
def clean_reviews():
    global df_reviews
    global df_testset_clean
    global df_trainset_clean
    #Modify Reviews.csv
    df_reviews["AuthorId"] = df_reviews["AuthorId"].astype("category")
    df_reviews = df_reviews.drop(columns={"Rating"})
    #Create Test und Trainset
    NAValues_Likes = df_reviews["Like"].isna()
    df_testset_reviews = df_reviews[NAValues_Likes]
    df_trainset_reviews = df_reviews[~NAValues_Likes]
    #Modify Trainset
    df_trainset_reviews["Like"] = df_trainset_reviews["Like"].astype(int)
    df_trainset_reviews = df_trainset_reviews.drop(columns={"TestSetId"})
    #Modify Testset
    df_testset_reviews["Like"] = df_testset_reviews["Like"].fillna(0)
    df_testset_reviews["Like"] = df_testset_reviews["Like"].astype(int)
    #Save Testset und Trainset
    df_trainset_clean = df_testset_reviews
    df_trainset_clean = df_trainset_reviews
    df_testset_reviews.to_csv("cleaned_testset.csv", index=False)
    df_trainset_reviews.to_csv("cleaned_trainset.csv", index=False)

#Clean Requests.csv
# --> Es gibt 140.195 rows, keine Spalte hat fehlende Werte
# --> Time-Attribut hat grosse Outlier
# --> Verteilung der Flags: 
    #High Calories: 0.0 = 83.806, 1.0 = 56.389; 
    #High Protein: Indifferent = 84.244, Yes = 55.951
    #High Fiber: 0 = 83.956, 1 = 56.239
    #Low Sugar: 0 = 98.113, Indifferent = 42.082
    #Low Fat: 0 = 98.209, 1 = 41.986
def clean_requests():
    global df_requests
    global df_requests_clean
    #Kategorisierung der Flags
    df_requests["HighCalories"] = df_requests["HighCalories"].astype("category")
    df_requests["HighProtein"] = df_requests["HighProtein"].astype("category")
    df_requests["LowFat"] = df_requests["LowFat"].astype("category")
    df_requests["LowSugar"] = df_requests["LowSugar"].astype("category")
    df_requests["HighFiber"] = df_requests["HighFiber"].astype("category")
    df_requests["AuthorId"] = df_requests["AuthorId"].astype("category")
    #Entfernen von Negativen Time-Werten
    df_requests = df_requests[df_requests["Time"]>0]
    #Entfernen von Outliern bei Time-Attribut
    Q1 = df_requests["Time"].quantile(0.25)
    Q3 = df_requests["Time"].quantile(0.75)
    IQR = Q3-Q1
    df_requests = df_requests[~((df_requests["Time"]>(Q3+3*IQR)))]
    #Save Request-Set
    df_requests_clean = df_requests
    df_requests.to_csv("cleaned_requests.csv", index=False)
    

#Clean Recipes
# --> Es gibt 75604 rows, RecipeServings und RecipeYields haben fehlende Werte
# --> CookTime und PrepTime haben grosse Outlier
# --> Fast jede XContent Spalte hat grosse Outlier, insbesondere Sodium, Cholesterol, Calories, Fat und Protein
# --> RecipeCategory Attribut ist wenig hilfreich, da extrem viele Werte den Wert 'Other' haben. Konkret:
    #Beverages         2.303
    #Bread             4.246
    #Breakfast         3.033
    #Lunch             4.887
    #One dish meal     4.590
    #Other            56.347
    #Soup               .198
# --> Hohe Korrelationen unter Content Werten
def clean_recipes():
    global df_recipes
    global df_recipes_clean
    #1. Entfernen Outliers CookTime
    
    Q1_CookTime = df_recipes["CookTime"].quantile(0.25)
    Q3_CookTime = df_recipes["CookTime"].quantile(0.75)
    IQR = Q3_CookTime-Q1_CookTime
    df_recipes = df_recipes[~((df_recipes["CookTime"]>((Q3_CookTime+3*IQR))))]
    
    #2. Entfernen Outliers PrepTime
    Q1_PrepTime = df_recipes["PrepTime"].quantile(0.25)
    Q3_PrepTime = df_recipes["PrepTime"].quantile(0.75)
    IQR_PrepTime = Q3_PrepTime -Q1_PrepTime
    df_recipes = df_recipes[~((df_recipes["PrepTime"]>(Q3_PrepTime+3*IQR_PrepTime)))]
    
    #3. Entfernen von falschen Content-Werten fuer Makronaehrstoffe
    
    max_Cal_per_100g = 900
    max_Fat_per_100g = 900
    max_Carbohydrates_per_100g = 420
    max_Proteins_per_100g = 420
    df_recipes = df_recipes[~((df_recipes["Calories"]>max_Cal_per_100g))]
    df_recipes = df_recipes[~((df_recipes["FatContent"]>max_Fat_per_100g))]
    df_recipes = df_recipes[~((df_recipes["CarbohydrateContent"]>max_Carbohydrates_per_100g))]
    df_recipes = df_recipes[~((df_recipes["SugarContent"]>max_Carbohydrates_per_100g))]
    df_recipes = df_recipes[~((df_recipes["ProteinContent"]>max_Proteins_per_100g))]
    
    
    #4. Entfernen von Outliers fuer Content-Werte von Mikronaehrstoffen
    #Outlier Removal SaturatedFatContent
    
    Q1_SatFat = df_recipes["SaturatedFatContent"].quantile(0.25)
    Q3_SatFat = df_recipes["SaturatedFatContent"].quantile(0.75)
    IQR_SatFat = Q3_SatFat-Q1_SatFat
    df_recipes = df_recipes[~((df_recipes["SaturatedFatContent"]>((Q3_SatFat+3*IQR_SatFat))))]
    #Outlier Removal CholesterolContent
    Q1_Cholesterol = df_recipes["CholesterolContent"].quantile(0.25)
    Q3_Cholesterol = df_recipes["CholesterolContent"].quantile(0.75)
    IQR_Cholesterol = Q3_Cholesterol-Q1_Cholesterol
    df_recipes = df_recipes[~((df_recipes["CholesterolContent"]>((Q3_Cholesterol+3*IQR_Cholesterol))))]
    #Outlier Removal SodiumContent
    Q1_Sodium = df_recipes["SodiumContent"].quantile(0.25)
    Q3_Sodium = df_recipes["SodiumContent"].quantile(0.75)
    IQR_Sodium = Q3_Sodium-Q1_Sodium
    df_recipes = df_recipes[~((df_recipes["SodiumContent"]>((Q3_Sodium+3*IQR_Sodium))))]
    #Outlier Removal Fiber
    Q1_Fiber = df_recipes["FiberContent"].quantile(0.25)
    Q3_Fiber = df_recipes["FiberContent"].quantile(0.75)
    IQR_Fiber = Q3_Fiber-Q1_Fiber
    df_recipes = df_recipes[~((df_recipes["FiberContent"]>((Q3_Fiber+3*IQR_Fiber))))]
    
    #5. Hinzufuegen eines Rezepttypes -> Omnivore, Vegan oder Vegetarisch
    erstelle_Meal_Typen()

    #6. Fuellen von NA Werten in RecipeServings
    df_recipes["RecipeServings"] = df_recipes["RecipeServings"].fillna(0)

    #7. Anpassen der RecipeYield Spalte
    df_recipes["RecipeYield"] = df_recipes["RecipeYield"].fillna("Undefined")
    df_recipes["RecipeYield"] = df_recipes["RecipeYield"].apply(translate_recipe_yields_to_categories)
    df_recipes["RecipeYield"] = df_recipes["RecipeYield"].astype("category")

    #8. Entfernen von unnoetigen Spalten
    df_recipes = df_recipes.drop(columns={"Name", "RecipeIngredientQuantities", "RecipeIngredientParts"})
    
    #9. Kategorisierung der letzten Attribute
    df_recipes["RecipeCategory"] = df_recipes["RecipeCategory"].astype("category")
    df_recipes["Type_Meal"] = df_recipes["Type_Meal"].astype("category")

    #10. Save Recipe-Set
    df_recipes_clean = df_recipes
    df_recipes.to_csv("cleaned_recipes.csv", index=False)
    
#-------------- Anpassen finales Dataset -------------------------#
def modify_final_dataset():
    global df_final
    ##Encoding von Categorical Variables
    df_final = df_final.drop(columns={"AuthorId", "RecipeId"})
    non_numeric_cols_train = df_final.select_dtypes(include=['object', 'category']).columns
    df_final = pd.get_dummies(df_final, columns=non_numeric_cols_train, drop_first=True)
    #Likes nach hinten
    likes = df_final["Like"]
    df_final = df_final.drop(columns= {"Like"})
    df_final["Like"] = likes

#-------------- Mergen Datasets -------------------------#

#Laden der Cleanen Datasets
clean_diet()
clean_reviews()
clean_requests()
clean_recipes()


#Mergen der Datasets
df_1 = pd.merge(df_diet_clean, df_trainset_clean, on = "AuthorId", how = "inner")
df_2 = pd.merge(df_1, df_recipes_clean, on= "RecipeId", how="inner")
df_final = pd.merge(df_2, df_requests_clean, on =["AuthorId", "RecipeId"], how= "inner")
df_final.to_csv("cleaned_final.csv", index=False)

modify_final_dataset()
#print(df_final)

def build_model():
    train_df, test_df = train_test_split(df_final, test_size=0.20, stratify=df_final['Like'], random_state=2023+2024)
    X_train = train_df.drop(columns={"Like"})
    y_train = train_df["Like"]
    X_test = test_df.drop(columns={"Like"})
    y_test = test_df["Like"]
    smote = SMOTE(random_state=42)
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
    # Modelle definieren
    model_random_forest = RandomForestClassifier()
    model_gradient_boosting = GradientBoostingClassifier()
    model_logistic_regression = LogisticRegression(max_iter=30)
    model_naive_bayes = GaussianNB()

    # Pipeline ohne PCA
    pipeline = Pipeline(steps=[("model", model_random_forest)])
    

    # Parameter Grid nur für RandomForestClassifier
    parameter_grid_random_forest = {
    "model__n_estimators": [10, 20, 50],  # Anzahl der Bäume im Wald
    "model__max_depth": [2, 3, 4], 
    "model__max_features": [3,5,20] # maximale Tiefe der Bäume
    #"model__n_estimators": [50],  # Anzahl der Bäume im Wald
    #"model__max_depth": [4], 
    #"model__max_features": [5] # maximale Tiefe der Bäume
    }
    # Grid Search Setup
    search = GridSearchCV(pipeline,
                      parameter_grid_random_forest, 
                      scoring="balanced_accuracy",
                      n_jobs=2, 
                      cv=5,  # Anzahl der Folds für die Kreuzvalidierung
                      error_score="raise"
    )
    # Training und Grid Search
    search.fit(X_train_smote, y_train_smote)
    # Beste Parameter ausgeben
    
    print("Beste Parameter:", search.best_params_, "(CV Score=%0.3f)" % search.best_score_)
    probabilities = search.predict_proba(X_test)
    threshold= 0.15
    predicted_classes = (probabilities[:, 1] >= threshold).astype(int)
    print(balanced_accuracy_score(y_test, predicted_classes))
    ct = pd.crosstab(predicted_classes, y_test,
                 rownames=["pred"], colnames=["true"])
    print(ct)

    # evaluate performance of model on test set
    print("Score on test set:", search.score(X_test, y_test))
    # contingency table
    ct = pd.crosstab(search.best_estimator_.predict(X_test), y_test,
                 rownames=["pred"], colnames=["true"])
    print(ct)

build_model()



C:\Users\Felix Puetterich\AppData\Local\Temp\ipykernel_10956\2575529016.py:31: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_reviews = pd.read_csv(file_path_reviews)
C:\Users\Felix Puetterich\AppData\Local\Temp\ipykernel_10956\2575529016.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trainset_reviews["Like"] = df_trainset_reviews["Like"].astype(int)
C:\Users\Felix Puetterich\AppData\Local\Temp\ipykernel_10956\2575529016.py:140: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting'

Beste Parameter: {'model__max_depth': 4, 'model__max_features': 3, 'model__n_estimators': 50} (CV Score=0.819)
0.5
true      0     1
pred             
1     13150  1734
Score on test set: 0.6541695282452055
true      0    1
pred            
0     10842  895
1      2308  839
